In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv("data/acled.csv")
df.head()

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp
0,BFO12795,2024-11-08,2024,1,Political violence,Violence against civilians,Attack,JNIM: Group for Support of Islam and Muslims,NaN,Rebel group,...,Bane,11.5756,-0.3566,1,Facebook,New media,"On 8 November 2024, JNIM militants attacked th...",2,NaN,1731368104
1,BFO12802,2024-11-08,2024,1,Strategic developments,Strategic developments,Other,JNIM: Group for Support of Islam and Muslims,NaN,Rebel group,...,Silmangue,13.8886,-0.6095,1,Facebook; Undisclosed Source,Local partner-Other,"Other: On 8 November 2024, JNIM militants cond...",0,NaN,1731368104
2,BFO12804,2024-11-08,2024,1,Political violence,Battles,Armed clash,JNIM: Group for Support of Islam and Muslims,NaN,Rebel group,...,Gorgadji,14.0329,-0.5197,1,Undisclosed Source,Local partner-Other,"On 8 November 2024, JNIM militants attacked a ...",0,NaN,1731368104
3,BOL6732,2024-11-08,2024,1,Political violence,Riots,Mob violence,Rioters (Bolivia),MAS-IPSP: Movement for Socialism-Political Ins...,Rioters,...,La Paz,-16.4957,-68.1336,1,Correo del Sur; El Deber; La Patria en Linea; ...,Subnational-National,"On 8 November 2024, in La Paz (La Paz), a grou...",0,crowd size=no report,1731368105
4,BOL6733,2024-11-08,2024,1,Demonstrations,Protests,Peaceful protest,Protesters (Bolivia),Teachers (Bolivia); CTEUB: Confederation of Ur...,Protesters,...,La Paz,-16.4957,-68.1336,1,El Diario (Bolivia),National,"On 8 November 2024, in La Paz (La Paz), a grou...",0,crowd size=no report,1731368105


In [48]:
#df.columns
df.tags.value_counts()
#df.admin1.value_counts()
#df.admin3.value_counts()

tags
crowd size=no report                               675702
crowd size=hundreds                                 26977
crowd size=dozens                                   23241
crowd size=large                                    12377
local administrators                                12119
                                                    ...  
counter-demonstration; crowd size=at least 230          1
counter-demonstration; crowd size=about 127-132         1
crowd size=between 1,600 and 1,640                      1
crowd size=at least 2,100                               1
crowd size=at least 35; sexual violence                 1
Name: count, Length: 13906, dtype: int64

In [ ]:
##key
df["keys"] = df["tags"].str.extract(r'^(.*?)(?=;)')
df["keys"] = df["keys"].str.replace(r'=.*$', '', regex=True)
df["keys"].value_counts()

keys
crowd size                                                         5405
counter-demonstration                                              2757
armed                                                               786
schools                                                             451
local administrators                                                398
count every vote                                                    161
armed presence                                                      154
women targeted: girls                                               136
car ramming                                                         117
sexual violence                                                      54
women targeted: activists/human rights defenders/social leaders      37
women targeted: political party supporters                           14
women targeted: candidates for office                                 7
women targeted: accused of witchcraft/sorcery              

In [5]:
type(df.tags[0])

float

In [10]:
# Create the 'no_info' column based on the conditions
df["no_info"] = df["tags"].apply(lambda x: 1 if pd.isna(x) or x == "crowd size=no report" else 0)

In [11]:
df.tags.value_counts()

tags
crowd size=no report                               675702
crowd size=hundreds                                 26977
crowd size=dozens                                   23241
crowd size=large                                    12377
local administrators                                12119
                                                    ...  
counter-demonstration; crowd size=at least 230          1
counter-demonstration; crowd size=about 127-132         1
crowd size=between 1,600 and 1,640                      1
crowd size=at least 2,100                               1
crowd size=at least 35; sexual violence                 1
Name: count, Length: 13906, dtype: int64

In [26]:
import re

# Function to classify crowd size
def classify_crowd_size(tag):
    if isinstance(tag, str) and re.search(r'\d', tag):  # Check if there's a digit in the tag
        # Extract the numeric value (handle commas)
        value = re.search(r'\d[\d,]*', tag).group()  # Match numbers with optional commas
        value = int(value.replace(',', ''))  # Remove commas and convert to integer
        # Classify based on value
        if value < 100:
            return "dozens"
        elif value < 1000:
            return "hundreds"
        else:
            return "large"
    elif isinstance(tag, str) and "no report" in tag:
        return "no report"
    return "other tag"

# Apply the classification logic
df["crowd_size"] = df["tags"].apply(classify_crowd_size)

In [29]:
df[df["tags"] == "counter-demonstration; crowd size=at least 230"]

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,longitude,geo_precision,source,source_scale,notes,fatalities,tags,timestamp,no_info,crowd_size
711565,USA49248,2022-10-26,2022,1,Demonstrations,Riots,Violent demonstration,Rioters (United States),Women (United States),Rioters,...,-122.4442,1,News Tribune,Subnational,"On 26 October 2022, about 30 supporters of Sta...",0,counter-demonstration; crowd size=at least 230,1680633816,0,hundreds


In [31]:
# List of values to create columns for
keywords = [
    "counter-demonstration",
    "armed",
    "schools",
    "local administrators",
    "count every vote",
    "armed presence",
    "women targeted: girls",
    "car ramming",
    "sexual violence",
    "women targeted: activists/human rights defenders/social leaders",
    "women targeted: political party supporters",
    "women targeted: candidates for office",
    "women targeted: accused of witchcraft/sorcery",
    "women targeted: government officials",
    "women targeted: politicians"
]

# Create a column for each keyword
for keyword in keywords:
    df[keyword] = df["tags"].apply(lambda x: 1 if isinstance(x, str) and keyword in x else 0)

In [35]:
df.columns

Index(['event_id_cnty', 'event_date', 'year', 'time_precision',
       'disorder_type', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'civilian_targeting', 'iso', 'region', 'country',
       'admin1', 'admin2', 'admin3', 'location', 'latitude', 'longitude',
       'geo_precision', 'source', 'source_scale', 'notes', 'fatalities',
       'tags', 'timestamp', 'no_info', 'crowd_size', 'counter-demonstration',
       'armed', 'schools', 'local administrators', 'count every vote',
       'armed presence', 'women targeted: girls', 'car ramming',
       'sexual violence',
       'women targeted: activists/human rights defenders/social leaders',
       'women targeted: political party supporters',
       'women targeted: candidates for office',
       'women targeted: accused of witchcraft/sorcery',
       'women targeted: government officials', 'women targeted: politicians'],
      dtype='object')

In [36]:
df[df["women targeted: accused of witchcraft/sorcery"] == 1]

,event_id_cnty,event_date,year,time_precision,disorder_type,event_type,sub_event_type,actor1,assoc_actor_1,inter1,...,armed presence,women targeted: girls,car ramming,sexual violence,women targeted: activists/human rights defenders/social leaders,women targeted: political party supporters,women targeted: candidates for office,women targeted: accused of witchcraft/sorcery,women targeted: government officials,women targeted: politicians
4028,DRC33950,2024-11-04,2024,1,Political violence,Riots,Mob violence,Rioters (Democratic Republic of Congo),Vigilante Group (Democratic Republic of Congo),Rioters,...,0,0,0,0,0,0,0,1,0,0
13045,CEN50083,2024-10-26,2024,2,Political violence,Riots,Mob violence,Rioters (Central African Republic),Vigilante Group (Central African Republic),Rioters,...,0,0,0,0,0,0,0,1,0,0
18517,DRC33901,2024-10-22,2024,1,Political violence,Riots,Mob violence,Rioters (Democratic Republic of Congo),Vigilante Group (Democratic Republic of Congo),Rioters,...,0,0,0,0,0,0,0,1,0,0
28349,SAF22227,2024-10-13,2024,1,Political violence,Riots,Mob violence,Rioters (South Africa),Vigilante Group (South Africa),Rioters,...,0,0,0,0,0,0,0,1,0,0
30974,IND169524,2024-10-10,2024,1,Political violence,Riots,Mob violence,Rioters (India),Vigilante Group (India),Rioters,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260112,DRC5947,2012-01-23,2012,1,Political violence,Riots,Mob violence,Rioters (Democratic Republic of Congo),Vigilante Group (Democratic Republic of Congo),Rioters,...,0,0,0,0,0,0,0,1,0,0
2279049,GHA112,2010-11-29,2010,1,Political violence,Riots,Mob violence,Rioters (Ghana),NaN,Rioters,...,0,0,0,0,0,0,0,1,0,0
2293549,SAF1660,2009-10-17,2009,1,Political violence,Riots,Mob violence,Rioters (South Africa),Vigilante Group (South Africa),Rioters,...,0,0,0,0,0,0,0,1,0,0
2308600,SAF839,2005-11-15,2005,3,Political violence,Riots,Mob violence,Rioters (South Africa),Vigilante Group (South Africa),Rioters,...,0,0,0,0,0,0,0,1,0,0
